In [1]:
import os 

import nltk 
# nltk.download('stopwords')
import numpy as np
import pandas as pd

#Gensim 
import gensim 
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#Spacy
import spacy 

#NLTK
from nltk.corpus import stopwords

#Vis
import pyLDAvis
import pyLDAvis.gensim_models 

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
old_dir = os.getcwd()
os.chdir("S:/asanchez/Edward Jones Stuff")

# read in data into dataframe
df = pd.read_excel('InsightExchange_AllQuestionText_misc_removed.xlsx')
df_deduped = df['QuestionText'].drop_duplicates()

df_deduped.head()

0    Approximately how much time do you spend on th...
1    What is the primary reason that you have <u>no...
2    To what extent do you find these communication...
3    Please indicate how much you agree or disagree...
4    When searching for <b>your Edward Jones' Finan...
Name: QuestionText, dtype: object

In [3]:
stopwords = stopwords.words("english")

In [4]:
df_deduped[0][0:90]

'Approximately how much time do you spend on the internet <u>daily</u> (including web brows'

In [5]:
def lemmatization(texts, allowed_tags = ["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_lg", disable = ["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_tags and token.is_oov==False:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return(texts_out)

In [6]:
lemmatized_texts = lemmatization(df_deduped)
print(lemmatized_texts[0][0:90])

C:\Users\asanchez\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_lg' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


approximately how much time do spend internet include web browser mobile app


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc = True) # remove accents if there
        final.append(new)
    return final

In [8]:
data_words = gen_words(lemmatized_texts)

In [9]:
print(data_words[0][0:20])

['approximately', 'how', 'much', 'time', 'do', 'spend', 'internet', 'include', 'web', 'browser', 'mobile', 'app']


In [10]:
# bigrams and trigrams 
bigram_phrases = gensim.models.Phrases(data_words, min_count = 5, threshold = 100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold = 100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)                                    
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# create a function to reinject above so model can process them
def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

In [11]:
# TF-IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(corpus, id2word=id2word) # create tfidf model

low_value = 0.03
words = []
words_missing_in_tfidf = []

# goes through and looks for words that occur with n threshold (low value) if that occurs 
# then it doesnt lend value to clustering algorithm and is excluded 
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids=[id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # the words with tfidf score 0 will be missing
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf] 
    corpus[i] = new_bow

In [12]:
''' without tfidf removal 
id2word = corpora.Dictionary(data_words)
corpus = []
for text in data_words:
    new = id2word.doc2bow(text) #create bag of words
    corpus.append(new)
'''

' without tfidf removal \nid2word = corpora.Dictionary(data_words)\ncorpus = []\nfor text in data_words:\n    new = id2word.doc2bow(text) #create bag of words\n    corpus.append(new)\n'

In [13]:
'''print(corpus[0][0:20])
word = id2word[[0][:1][0]]'''

'print(corpus[0][0:20])\nword = id2word[[0][:1][0]]'

In [14]:
'''
#checking to make sure word map makes sense
print(word)'''

'\n#checking to make sure word map makes sense\nprint(word)'

In [15]:
'''# create LDA model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus = corpus, 
    id2word = id2word, 
    num_topics = 30, # number of topics 
    random_state = 100, 
    update_every = 1, 
    chunksize=100, 
    passes=10, 
    alpha = "auto"
)'''

# create LDA model for train_test_split
lda_model = gensim.models.ldamodel.LdaModel(
    corpus = corpus[:-1], # training model to everything except second to last item in corpus 
    id2word = id2word, 
    num_topics = 25, # number of topics 
    random_state = 100, 
    update_every = 1, 
    chunksize=100, 
    passes=10, 
    alpha = "auto"
)

In [16]:
# testing model with unseen data 
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)
# different documents and degree to which document falls into that catagory
# organized in ascending order from topic 

[(0, 0.087858364), (2, 0.025184842), (3, 0.022488466), (5, 0.16575836), (7, 0.02424734), (8, 0.015853774), (10, 0.08912644), (12, 0.012440146), (13, 0.23622794), (15, 0.020968262), (16, 0.010925544), (18, 0.0132486), (19, 0.09862222), (21, 0.013852575), (23, 0.08812964), (24, 0.013483859)]


In [17]:
# organize list by most represented topic 
# sort list of lists
def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)

new_vector = Sort(vector)
new_vector

[(13, 0.23622794),
 (5, 0.16575836),
 (19, 0.09862222),
 (10, 0.08912644),
 (23, 0.08812964),
 (0, 0.087858364),
 (2, 0.025184842),
 (7, 0.02424734),
 (3, 0.022488466),
 (15, 0.020968262),
 (8, 0.015853774),
 (21, 0.013852575),
 (24, 0.013483859),
 (18, 0.0132486),
 (12, 0.012440146),
 (16, 0.010925544)]

In [18]:
'''#Saving the model (one way)
os.mkdir('models')
lda_model.save('models/test_model.model') '''

"#Saving the model (one way)\nos.mkdir('models')\nlda_model.save('models/test_model.model') "

In [19]:
# Vusalizing the Data

In [20]:
pyLDAvis.enable_notebook()

In [21]:
vis = pyLDAvis.gensim_models.prepare(lda_model, 
                                     corpus, 
                                     id2word, 
                                     mds="mmds", 
                                     R=30)


C:\Users\asanchez\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [22]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.223617  0.485620       1        1  19.828054
7      0.439165 -0.022212       2        1   7.021293
19    -0.449315 -0.128607       3        1   5.653083
2      0.416143  0.113283       4        1   5.608744
15    -0.444157  0.001263       5        1   5.313371
3     -0.288736 -0.183816       6        1   5.151643
10     0.305848 -0.096458       7        1   5.059265
8     -0.384477  0.140238       8        1   3.915415
24     0.263068 -0.366764       9        1   3.876236
21     0.403361 -0.255552      10        1   3.439110
16     0.310048  0.268454      11        1   3.289304
18     0.219704  0.112539      12        1   3.200728
5      0.120588 -0.383332      13        1   3.143135
0     -0.267081 -0.366259      14        1   3.051554
23    -0.136726 -0.358660      15        1   2.753054
12    -0.169057  0.093502      16        1   2.633337
22    -0.301941  0.287843      17        1   2.482563
4      0.088427  0.260512      18        1   2.333186
17     0.032652 -0.181079      19        1   2.282857
9      0.005805  0.420145      20        1   2.205820
6     -0.093104  0.273154      21        1   2.118924
11    -0.199467  0.372909      22        1   2.033528
1     -0.126970 -0.104889      23        1   1.842927
20    -0.007019 -0.375483      24        1   1.411735
14     0.039622 -0.006351      25        1   0.351130, topic_info=         Term         Freq        Total Category  logprob  loglift
12         be  2300.000000  2300.000000  Default  30.0000  30.0000
44    account   384.000000   384.000000  Default  29.0000  29.0000
14       have   576.000000   576.000000  Default  28.0000  28.0000
329     video   354.000000   354.000000  Default  27.0000  27.0000
4         how   750.000000   750.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
454       see     0.021969     1.292245  Topic25  -8.1752   1.5773
379       set     0.021969     1.292239  Topic25  -8.1752   1.5773
922       own     0.021969     1.292240  Topic25  -8.1752   1.5773
507    screen     0.021969     1.292246  Topic25  -8.1752   1.5773
887  download     0.021969     1.292237  Topic25  -8.1752   1.5773

[957 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
43       17  0.993064      access
114      15  0.712448  accomplish
44        2  0.132612     account
44       12  0.863279     account
204      10  0.466623     achieve
...     ...       ...         ...
40        8  0.989783         why
296       1  0.924528      window
198       4  0.988835        work
340      20  0.749253       worry
201      17  0.370968         zip

[375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 8, 20, 3, 16, 4, 11, 9, 25, 22, 17, 19, 6, 1, 24, 13, 23, 5, 18, 10, 7, 12, 2, 21, 15])

In [82]:
'''# loading model
new_model = gensim.models.ldamodel.LdaModel.load('models/test_model.model')'''